In [2]:
import jenkspy

In [ ]:
df = pd.read_csv('dissimilarity_table.csv')

In [3]:
list_of_values = []

breaks = jenkspy.jenks_breaks(list_of_values, nb_class=6)

ValueError: Number of class have to be an integer greater than 2 and smaller than the number of values to use